# Chars-by text generation: Russian chastushki


In [6]:
import sys
import random
import numpy as np

from keras.callbacks import *
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop

try:
    f = open('data/chastushki.txt', encoding='windows-1251')
    text = f.read().lower()
    
    start = text.index('*')
    finish = len(text) - text[::-1].index('*')

    text = text[start:finish]   

    print('Corpus length:', len(text))
    print('Text example: \n', text[:300])

except:
    print('File is not valid or does not exist')

Corpus length: 306175
Text example: 
 * * *

		«сколько шуму на селе:
		знать, мужики дерутся?»
		нет, укрепляются в земле,
		между собой грызутся.


* * *

		«что вас, девки, только две,
		а третья?-то у вас где?»
		у нас третья-то пропала,
		во саратов жить попала.


* * *

		акулина тетушка,
		тяжела работушка.
		всё покосы, нивушка,


# Vectorize

In [2]:
chars = sorted(list(set(text)))
print('Total chars used:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


maxlen = 32 # in chars
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('Sentences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print('Done!')    

Total chars used: 67
Sentences: 102048
Vectorization...
Done!


# Do some function-declare shit

In [3]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    example_file.write('\n Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    example_file.write('\nGenerating with seed: "' + sentence + '"')
    example_file.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 1)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        example_file.write(next_char)
        example_file.flush()


# Build and fit model

In [4]:
from keras.layers import Bidirectional, Dropout

print('Building model...')
model = Sequential()
model.add(Bidirectional(LSTM(128, input_shape=(maxlen, len(chars)))))
model.add(Dropout(rate=0.2))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done!')

Building model...
Done!


In [7]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

example_file = open('output/chastushki example.txt', 'w')

print('Fitting model...')
history = model.fit(x, y,
                    batch_size=256,
                    epochs=1500,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[print_callback, reduce_lr])

Fitting model...
Train on 91843 samples, validate on 10205 samples
Epoch 1/1500
91843/91843 [==============================] - 135s 1ms/step - loss: 2.2204 - val_loss: 2.6233
Epoch 2/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 1.8120 - val_loss: 2.4563
Epoch 3/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 1.6905 - val_loss: 2.4446
Epoch 4/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 1.6044 - val_loss: 2.4368
Epoch 5/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 1.5418 - val_loss: 2.3705
Epoch 6/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 1.4897 - val_loss: 2.4605


c:\ivan\python36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 7/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 1.4473 - val_loss: 2.4742
Epoch 8/1500
91843/91843 [==============================] - 118s 1ms/step - loss: 1.4052 - val_loss: 2.5964
Epoch 9/1500
91843/91843 [==============================] - 118s 1ms/step - loss: 1.3693 - val_loss: 2.5674
Epoch 10/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 1.3341 - val_loss: 2.6048
Epoch 11/1500
91843/91843 [==============================] - 118s 1ms/step - loss: 1.1899 - val_loss: 2.7028
Epoch 12/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 1.1274 - val_loss: 2.7480
Epoch 13/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 1.0921 - val_loss: 2.8103
Epoch 14/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 1.0578 - val_loss: 2.8682
Epoch 15/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 1.0285 - val_loss: 2.9217
Epoch 16/1500
91843/91

91843/91843 [==============================] - 120s 1ms/step - loss: 0.6016 - val_loss: 4.0139
Epoch 83/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 0.5988 - val_loss: 4.0187
Epoch 84/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 0.5986 - val_loss: 4.0057
Epoch 85/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 0.5913 - val_loss: 4.0180
Epoch 86/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 0.5935 - val_loss: 4.0327
Epoch 87/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 0.5904 - val_loss: 4.0600
Epoch 88/1500
91843/91843 [==============================] - 118s 1ms/step - loss: 0.5884 - val_loss: 4.0424
Epoch 89/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5876 - val_loss: 4.0424
Epoch 90/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5891 - val_loss: 4.0421
Epoch 91/1500
91843/91843 [======

91843/91843 [==============================] - 120s 1ms/step - loss: 0.5235 - val_loss: 4.3950
Epoch 157/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 0.5139 - val_loss: 4.4458
Epoch 158/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 0.5168 - val_loss: 4.4085
Epoch 159/1500
91843/91843 [==============================] - 120s 1ms/step - loss: 0.5183 - val_loss: 4.4103
Epoch 160/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5137 - val_loss: 4.4330
Epoch 161/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5143 - val_loss: 4.4435
Epoch 162/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5132 - val_loss: 4.4423
Epoch 163/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.5159 - val_loss: 4.4434
Epoch 164/1500
91843/91843 [==============================] - 119s 1ms/step - loss: 0.5129 - val_loss: 4.4471
Epoch 165/1500
91843/9184

...

Epoch 1192/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.3160 - val_loss: 5.5009
Epoch 1193/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3106 - val_loss: 5.5508
Epoch 1194/1500
91843/91843 [==============================] - 126s 1ms/step - loss: 0.3159 - val_loss: 5.5455
Epoch 1195/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3081 - val_loss: 5.5380
Epoch 1196/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3106 - val_loss: 5.5426
Epoch 1197/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3122 - val_loss: 5.5211
Epoch 1198/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3128 - val_loss: 5.5693
Epoch 1199/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.3085 - val_loss: 5.5634
Epoch 1200/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.3137 - val_loss: 5.5631
E

Epoch 1266/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.3085 - val_loss: 5.6007
Epoch 1267/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.3049 - val_loss: 5.6028
Epoch 1268/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3043 - val_loss: 5.5680
Epoch 1269/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3021 - val_loss: 5.5650
Epoch 1270/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.3049 - val_loss: 5.5904
Epoch 1271/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.3063 - val_loss: 5.5636
Epoch 1272/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3099 - val_loss: 5.5769
Epoch 1273/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3107 - val_loss: 5.5951
Epoch 1274/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3056 - val_loss: 5.6208
E

Epoch 1340/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3031 - val_loss: 5.6064
Epoch 1341/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.3056 - val_loss: 5.5901
Epoch 1342/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3047 - val_loss: 5.5830
Epoch 1343/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.3045 - val_loss: 5.6017
Epoch 1344/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.3042 - val_loss: 5.5410
Epoch 1345/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.2984 - val_loss: 5.6296
Epoch 1346/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3021 - val_loss: 5.6201
Epoch 1347/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.3024 - val_loss: 5.5900
Epoch 1348/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.3014 - val_loss: 5.6233
E

Epoch 1414/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2980 - val_loss: 5.6504
Epoch 1415/1500
91843/91843 [==============================] - 125s 1ms/step - loss: 0.2960 - val_loss: 5.6430
Epoch 1416/1500
91843/91843 [==============================] - 129s 1ms/step - loss: 0.2976 - val_loss: 5.6476
Epoch 1417/1500
91843/91843 [==============================] - 129s 1ms/step - loss: 0.2959 - val_loss: 5.6422
Epoch 1418/1500
91843/91843 [==============================] - 128s 1ms/step - loss: 0.2982 - val_loss: 5.6446
Epoch 1419/1500
91843/91843 [==============================] - 130s 1ms/step - loss: 0.2968 - val_loss: 5.6612
Epoch 1420/1500
91843/91843 [==============================] - 131s 1ms/step - loss: 0.2957 - val_loss: 5.6433
Epoch 1421/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.2948 - val_loss: 5.6159
Epoch 1422/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.2939 - val_loss: 5.6141
E

Epoch 1488/1500
91843/91843 [==============================] - 123s 1ms/step - loss: 0.2941 - val_loss: 5.6612
Epoch 1489/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2915 - val_loss: 5.6860
Epoch 1490/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.2916 - val_loss: 5.6448
Epoch 1491/1500
91843/91843 [==============================] - 124s 1ms/step - loss: 0.2917 - val_loss: 5.6642
Epoch 1492/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2915 - val_loss: 5.6517
Epoch 1493/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2946 - val_loss: 5.6567
Epoch 1494/1500
91843/91843 [==============================] - 121s 1ms/step - loss: 0.3011 - val_loss: 5.6558
Epoch 1495/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2925 - val_loss: 5.6771
Epoch 1496/1500
91843/91843 [==============================] - 122s 1ms/step - loss: 0.2947 - val_loss: 5.6857
E